# Implementing an Iris classifier

### Imports

In [ ]:
import os
import pickle
import shutil
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

np.random.seed(99)

## Model development

### Load dataset

In [ ]:
def iris_to_id(name: str):
    if name == b'Iris-setosa':
        return 0.
    elif name == b'Iris-versicolor':
        return 1.
    elif name == b'Iris-virginica':
        return 2.
    raise Exception(f"Unknown iris {name}")

data = np.loadtxt("iris.csv", delimiter=',', converters={4: iris_to_id})

X = data[:,:4]
y = data[:,4].astype(np.int32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes = 3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 3)

### Train model

In [ ]:
def create_model() -> tf.keras.Model:
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(5, activation='relu', input_shape=(4, )))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.02, weight_decay=1e-6)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def train_model(x_values, y_values, epochs=50):
    model = create_model()
    model.fit(x_values, y_values, epochs=epochs, validation_split=0.2, batch_size=32, verbose=0)

    return model

In [ ]:
trained_model = train_model(X_train, y_train)

### Evaluate model

In [ ]:
score = trained_model.evaluate(X_test, y_test, verbose = 0)
print('Test accuracy:', score[1])

## Store model for AIYRA

In [ ]:
# constants
MODEL_DIR = "model"
ZIP_FILENAME = "model"

os.mkdir(MODEL_DIR)

In [ ]:
# save model in a folder
trained_model.save(MODEL_DIR, save_format="tf")

# save train data for model calibration
with open(os.path.join(MODEL_DIR, 'calibration.pkl'), 'wb') as file:
    pickle.dump(X_train, file)

# zip model folder
shutil.make_archive(ZIP_FILENAME, 'zip', MODEL_DIR)

## Drafts

### Evaluate on a single input

In [ ]:
xi = np.array([3.0, 4.0, 1.0, 1.0]).reshape(1, -1)
yi = np.array([0.636719, 0.296875, 0.070312]).reshape(1, -1)

yi_predicted = trained_model.predict(xi)

print("expected:", yi, "\npredicted:", yi_predicted)

score = trained_model.evaluate(xi, yi, verbose=0)
print("accuracy: ", score[1])